# Chatbot Development

In this tutorial, we will go over chatbot development. Chatbot requires multi-turn conversations i.e., model has to remember previous conversations. We will show how to setup system, user, and assistant prompts to make the model capable of seamless conversation.

In [1]:
!pip install -q openai gradio

In [2]:
from openai import OpenAI
from rich import print
import json

In [3]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

## Helper function for messages

In this function, we will use a list of messages instead of a single prompt like the previous tutorials.

In [4]:
def generate_response(messages, temperature: float =0.0, max_tokens:int = 50):

  try:
    # Create an OpenAI client using the API key
    client = OpenAI(api_key=openai_api_key)

    # Generate a response using the OpenAI chat completions API, specifying model, messages, temperature, and max_tokens
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
  except Exception as error:
    # Handle any exceptions that may occur during API usage and print the error message
    print(error)

  # Return the content of the generated response as a string
  return response.choices[0].message.content

## LLMs are stateless

Language models don't have any memory or remember the previous state by design. They are stateless. Let's demonstrate that with an example.

In [5]:
messages = []
prompt = "Hey there! My name is Santi!"
messages.append({"role": "user", "content": prompt})

response = generate_response(messages)
print(response)

Hi Santi! Nice to meet you. How can I assist you today?

In [6]:
messages = []
prompt = "Hey there! What is my name?"
messages.append({"role": "user", "content": prompt})

response = generate_response(messages)
print(response)

I'm sorry, I don't know your name. Can you please tell me?

## Keep track of history

In order to keep track of history, we need to append the previous user inputs and assistant responses and pass the entire thread to the model. This can be done by incrementally adding `user` prompt and `assistant` response to messages list and pass the messages list to the model.

In [7]:
messages = []
prompt = "Hey there! My name is Santi!"
messages.append({"role": "user", "content": prompt})

response = generate_response(messages)
messages.append({"role": "assistant", "content": response})

prompt = "Hey there! What is my name?"
messages.append({"role": "user", "content": prompt})

response = generate_response(messages)
print(response)

Your name is Santi.

In [8]:
print(messages)

[
    {'role': 'user', 'content': 'Hey there! My name is Santi!'},
    {'role': 'assistant', 'content': 'Hi Santi! Nice to meet you. How can I assist you today?'},
    {'role': 'user', 'content': 'Hey there! What is my name?'}
]

# Basic Chatbot

Let's build a basic chatbot to demonstrate this back and forth.

We will use [Gradio](https://www.gradio.app/docs/chatinterface) to build a basic chatbot.

Function `chatbot_response`:

a. takes a user message and conversation history as arguments.  
b. organizes the messages into a conversation format, generates a response and returns it.

A Gradio chat interface named demo is created, allowing users to interact with the chatbot, and it's launched for use.

In [9]:
import gradio as gr

def chatbot_response(message, history):

  # Create an empty list named messages to store the conversation history.
  messages = []

  # Iterate through each entry in the history list.
  for entry in history:

    # Append a dictionary to the messages list representing a user message with the content from entry[0].
    messages.append({"role" : "user", "content": entry[0]})

    # Append another dictionary to the messages list representing an assistant message with the content from entry[1].
    messages.append({"role" : "assistant", "content": entry[1]})

  # Append a new dictionary to the messages list representing the current user message with the content from the 'message' argument.
  messages.append({"role" : "user", "content" : message})

  # Call a function named generate_response with the 'messages' list as an argument to generate a response.
  response = generate_response(messages)

  # Return the generated response.
  return response

demo = gr.ChatInterface(chatbot_response)


demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fad7543dcef1ec7bd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Thai Restaurant Order Chatbot

Let's do something a bit more complex. Let's make a chatbot, named ThaiBot that can greet customers, collect detailed pizza orders, clarify preferences, handle additional items, and manage payments. For delivery orders, it will request the address. The bot will maintain a friendly, conversational style throughout interactions.

In order to build this bot, we will make a system prompt that has all the necessary information to perform its operations.

In [10]:
system_prompt = {'role':'system', 'content':"""
You are ThaiBot, an automated service for a Thai restaurant. \
You start by warmly greeting the customer, taking their order, and inquiring whether it's for dine-in or takeout. \
You patiently collect the entire order, summarize it, and double-check if there are any additional items the customer would like. If it's for delivery, kindly request the delivery address. Finally, you process the payment. Please ensure to clarify all menu options, extras, and portion sizes for a seamless ordering experience. \
Your responses should be brief, friendly, and conversational.

The menu features a variety of Thai dishes, including:

Pad Thai: 12.95, 10.00, 7.00
Green Curry: 10.95, 9.25, 6.50
Tom Yum Soup: 11.95, 9.75, 6.75
Spring Rolls: 4.50, 3.50
Thai Salad: 7.25
For customizing your dishes, we offer toppings such as:

Extra Tofu: 2.00
Fresh Basil: 1.50
Shrimp: 3.00
Chicken: 3.50
Spicy Sauce: 1.50
Chili Peppers: 1.00
We also have a selection of beverages:

Thai Iced Tea: 3.00
Coconut Water: 3.00
Bottled Water: 5.00
"""}

Function `chatbot_response`:

a. takes a user message and conversation history as arguments.  
b. organizes the messages into a conversation format, generates a response and returns it.

A Gradio chat interface named demo is created, allowing users to interact with the chatbot, and it's launched for use.

In [11]:
import gradio as gr

def order_chatbot_response(message, history):

  # Create an empty list named messages to store the conversation history.
  messages = []

  # Append the system_prompt (which should be predefined elsewhere) to the messages list.
  messages.append(system_prompt)

  # Iterate through each entry in the history list.
  for entry in history:
    # Append a dictionary to the messages list representing a user message with the content from entry[0].
    messages.append({"role" : "user", "content": entry[0]})
    # Append another dictionary to the messages list representing an assistant message with the content from entry[1].
    messages.append({"role" : "assistant", "content": entry[1]})

  # Append a new dictionary to the messages list representing the current user message with the content from the 'message' argument.
  messages.append({"role" : "user", "content" : message})

  # Call a function named generate_response with the 'messages' list as an argument to generate a response.
  response = generate_response(messages)

  # Return the generated response.
  return response

order_demo = gr.ChatInterface(order_chatbot_response)


order_demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://89551dcf33e45a5207.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
